# SET UP

In [20]:
# Import requried libraries 
import pandas as pd
import numpy as np
import time
from datetime import datetime
!pip install gspread
!pip install gspread_dataframe
from gspread_dataframe import set_with_dataframe
import gspread
gc = gspread.service_account(filename='world-wide-web-356017-564c77672260.json')

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


# Extract

In [21]:
# extract first df and create variable "prices"
prices = pd.read_csv('/work/cerebro/worldwide internet prices in 2022 - IN 2022.csv')

In [22]:
# extract second df and create variable "speed"
speed = pd.read_csv('/work/cerebro/worldwide internet speed in 2022  - avg speed.csv')

In [23]:
# extract third df and create variable "users"
users = pd.read_csv('/work/cerebro/worldwide internet users - users.csv')

In [4]:
# Checking prices columns, null values and data types
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Country code                                      241 non-null    object 
 1   Name                                              242 non-null    object 
 2   Continental region                                242 non-null    object 
 3   NO. OF Internet Plans                             230 non-null    float64
 4   Average price of 1GB (USD)                        242 non-null    object 
 5   Cheapest 1GB for 30 days (USD)                    230 non-null    object 
 6   Most expensive 1GB (USD)                          230 non-null    object 
 7   Average price of 1GB (USD  at the start of 2021)  230 non-null    object 
 8   Average price of 1GB (USD – at start of 2020)     230 non-null    object 
dtypes: float64(1), object(

In [57]:
prices['Average price of 1GB (USD)'] = prices['Average price of 1GB (USD)'].str.lstrip('$')

In [59]:
prices['Average price of 1GB (USD)'] = pd.to_numeric(prices['Average price of 1GB (USD)'])

ValueError: Unable to parse string "NO PROVIDERS" at position 230

In [5]:
prices.index[prices['NO. OF Internet Plans '].isnull(),].tolist()

[230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241]

In [55]:
prices.loc[230:241,'Name']

230                  Christmas Island
231                           Eritrea
232                       North Korea
233                  Marshall Islands
234           St. Pierre and Miquelon
235                       South Sudan
236                            Tuvalu
237           Vatican City (Holy See)
238                         Venezuela
239                 Wallis and Futuna
240    Congo (Democratic Republic of)
241                          Zimbabwe
Name: Name, dtype: object

In [53]:
prices.columns

Index(['Country code', 'Name', 'Continental region', 'NO. OF Internet Plans ',
       'Average price of 1GB (USD)', 'Cheapest 1GB for 30 days (USD)',
       'Most expensive 1GB (USD)',
       'Average price of 1GB (USD  at the start of 2021)',
       'Average price of 1GB (USD – at start of 2020)'],
      dtype='object')

In [52]:
speed

,Country,Avg \n(Mbit/s)Ookla
0,Canada,75.18
1,South Korea,117.95
2,Netherlands,108.33
3,Japan,44.05
4,Norway,134.73
...,...,...
139,Sudan,9.50
140,Cuba,9.10
141,Tajikistan,7.05
142,Venezuela,5.68


In [51]:
users

,Country or area,Subregion,Region,Internet users,Population
0,China,Eastern Asia,Asia,"1,010,740,000","1,427,647,786"
1,India,Southern Asia,Asia,"833,710,000","1,352,642,280"
2,United States,Northern America,Americas,"312,320,000","327,096,265"
3,Indonesia,South-eastern Asia,Asia,"196,000,000","267,670,543"
4,Brazil,South America,Americas,"160,010,801","209,469,323"
...,...,...,...,...,...
209,"Saint Helena, Ascension and Tristan da Cunha",Western Africa,Africa,"2,906","6,035"
210,Falkland Islands,South America,Americas,"2,881","3,234"
211,Montserrat,Caribbean,Americas,"2,833","4,993"
212,Wallis and Futuna,Polynesia,Oceania,"1,383","11,661"


In [45]:
prices_speed_users = pd.merge(prices,speed,left_on='Name',right_on='Country',how='inner')

In [50]:
# prices_speed_users.loc[prices_speed_users['Country'].isnull(),]
prices_speed_users

,Country code,Name,Continental region,NO. OF Internet Plans,Average price of 1GB (USD),Cheapest 1GB for 30 days (USD),Most expensive 1GB (USD),Average price of 1GB (USD at the start of 2021),Average price of 1GB (USD – at start of 2020),Country,Avg \n(Mbit/s)Ookla
0,IL,Israel,NEAR EAST,27.0,$0.05,$0.02,$20.95,$0.11,$0.90,Israel,28.01
1,KG,Kyrgyzstan,CIS (FORMER USSR),20.0,$0.15,$0.10,$7.08,$0.21,$0.27,Kyrgyzstan,16.30
2,FJ,Fiji,OCEANIA,18.0,$0.19,$0.05,$0.85,$0.59,$3.57,Fiji,25.99
3,IT,Italy,WESTERN EUROPE,29.0,$0.27,$0.09,$3.54,$0.43,$1.73,Italy,37.15
4,SD,Sudan,SUB-SAHARAN AFRICA,33.0,$0.27,$0.03,$0.92,$0.63,$0.68,Sudan,9.50
...,...,...,...,...,...,...,...,...,...,...,...
130,YE,Yemen,NEAR EAST,17.0,$15.98,$0.40,$39.94,$15.98,$15.73,Yemen,16.89
131,NaN,Namibia,SUB-SAHARAN AFRICA,33.0,$22.37,$1.20,$72.06,$4.78,$11.02,Namibia,18.43
132,MW,Malawi,SUB-SAHARAN AFRICA,5.0,$25.46,$20.37,$25.46,$27.41,$3.59,Malawi,17.95
133,VE,Venezuela,SOUTH AMERICA,NaN,HYPERINFLATION,NaN,NaN,NaN,NaN,Venezuela,5.68


In [49]:
speed

,Country,Avg \n(Mbit/s)Ookla
0,Canada,75.18
1,South Korea,117.95
2,Netherlands,108.33
3,Japan,44.05
4,Norway,134.73
...,...,...
139,Sudan,9.50
140,Cuba,9.10
141,Tajikistan,7.05
142,Venezuela,5.68


# Load

In [62]:
#load data to google sheets
#Create key
key = "1ta2iOwjKzIRu7CL60DqzXpVlYhvaTOz7618GmgF8Kts"

#Selects the sheet to export to
sheet = gc.open_by_key(key).worksheet('prices')
sheet1 = gc.open_by_key(key).worksheet('speed')
sheet2 = gc.open_by_key(key).worksheet('users')

#Clears the sheet and exports data
sheet.batch_clear(["A100:L40000"])
set_with_dataframe(sheet, prices) 

sheet1.batch_clear(["A100:R15000"])
set_with_dataframe(sheet1, speed) 

sheet2.batch_clear(["A100:Z15000"])
set_with_dataframe(sheet2, users)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=07a4ffe0-c8ed-4f4e-b576-dc2d51ba276d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>